In [ ]:
import glob
from teipy import TeiReader

## ad @ref to tei:author
* go through all TEIs
* in each author node, find the email node, slugify the value
* use this for the tei:author's @ref

In [ ]:
files = glob.glob("../dhd_*/TEI/*.xml")

In [ ]:
len(files)

In [ ]:
all_docs = []
for x in files:
    doc = TeiReader(x)
    [x for x in doc.add_refs()]
    doc.tree_to_file(file=x)